## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Description,Country,Date
0,0,Jamestown,42.0970,-79.2353,23.09,69,100,5.01,snow,US,2022-02-04 03:07:54
1,1,Bambous Virieux,-20.3428,57.7575,80.22,88,99,21.50,light rain,MU,2022-02-04 03:13:39
2,2,Espinosa,-14.9261,-42.8192,70.30,87,43,3.58,scattered clouds,BR,2022-02-04 03:13:40
3,3,Cherskiy,68.7500,161.3000,-28.86,99,16,3.36,few clouds,RU,2022-02-04 03:10:08
4,4,Udachnyy,66.4167,112.4000,-22.68,99,98,2.51,overcast clouds,RU,2022-02-04 03:12:25


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_city_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_city_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Description,Country,Date
2,2,Espinosa,-14.9261,-42.8192,70.30,87,43,3.58,scattered clouds,BR,2022-02-04 03:13:40
5,5,Atuona,-9.8000,-139.0333,79.00,72,4,17.02,clear sky,PF,2022-02-04 03:13:41
10,10,Coari,-4.0850,-63.1414,73.98,98,100,1.52,overcast clouds,BR,2022-02-04 03:13:42
13,13,Monrovia,6.3005,-10.7969,77.68,86,10,2.44,clear sky,LR,2022-02-04 03:10:01
16,16,Port Blair,11.6667,92.7500,78.84,78,40,10.36,mist,IN,2022-02-04 03:13:44
18,18,Luderitz,-26.6481,15.1594,68.16,70,1,15.68,clear sky,NaN,2022-02-04 03:10:49
23,23,Sao Filipe,14.8961,-24.4956,72.52,77,0,8.75,clear sky,CV,2022-02-04 03:12:53
28,28,Rio Grande,-32.0350,-52.0986,70.95,86,100,10.00,light rain,BR,2022-02-04 03:10:38
30,30,Cidreira,-30.1811,-50.2056,75.70,89,84,22.15,broken clouds,BR,2022-02-04 03:09:33
33,33,Senanga,-16.1167,23.2667,65.39,95,97,2.48,light rain,ZM,2022-02-04 03:13:49


In [7]:
# 4a. Determine if there are any empty rows.
preferred_city_df.count()

City_ID             262
City                262
Lat                 262
Lng                 262
Max Temp            262
Humidity            262
Cloudiness          262
Wind Speed          262
City Description    262
Country             255
Date                262
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_city_df.dropna()
clean_df.count()

City_ID             255
City                255
Lat                 255
Lng                 255
Max Temp            255
Humidity            255
Cloudiness          255
Wind Speed          255
City Description    255
Country             255
Date                255
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "City Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,City Description,Lat,Lng,Hotel Name
2,Espinosa,BR,70.30,scattered clouds,-14.9261,-42.8192,
5,Atuona,PF,79.00,clear sky,-9.8000,-139.0333,
10,Coari,BR,73.98,overcast clouds,-4.0850,-63.1414,
13,Monrovia,LR,77.68,clear sky,6.3005,-10.7969,
16,Port Blair,IN,78.84,mist,11.6667,92.7500,
23,Sao Filipe,CV,72.52,clear sky,14.8961,-24.4956,
28,Rio Grande,BR,70.95,light rain,-32.0350,-52.0986,
30,Cidreira,BR,75.70,broken clouds,-30.1811,-50.2056,
33,Senanga,ZM,65.39,light rain,-16.1167,23.2667,
37,Tocopilla,CL,68.99,few clouds,-22.0920,-70.1979,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,City Description,Lat,Lng,Hotel Name
2,Espinosa,BR,70.30,scattered clouds,-14.9261,-42.8192,Hotel Vilma
5,Atuona,PF,79.00,clear sky,-9.8000,-139.0333,Villa Enata
10,Coari,BR,73.98,overcast clouds,-4.0850,-63.1414,HOTEL ALEX FLAT
13,Monrovia,LR,77.68,clear sky,6.3005,-10.7969,Sunset Inn
16,Port Blair,IN,78.84,mist,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
23,Sao Filipe,CV,72.52,clear sky,14.8961,-24.4956,Hotel Xaguate
28,Rio Grande,BR,70.95,light rain,-32.0350,-52.0986,Hotel Atlântico Rio Grande
30,Cidreira,BR,75.70,broken clouds,-30.1811,-50.2056,Hotel Castelo
33,Senanga,ZM,65.39,light rain,-16.1167,23.2667,Zango lodge
37,Tocopilla,CL,68.99,few clouds,-22.0920,-70.1979,Hotel Galvarino


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "clean_data.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))